# Scenario Comparison Analysis

This notebook provides an interactive GUI for comparing two simulation scenarios side-by-side.

**Note:** Both scenarios being compared share the same topology and ISP subgraphs.
The comparison shows how different routing strategies (e.g., disaster-aware vs. standard routing) affect the same network.

## Features:

1. **Scenario Selection**: Load and compare any two scenarios from the output folder
2. **View Modes**:
   - **Topology View**: Compare ISP subgraphs in normal/disaster mode
   - **Statistics View**: Compare metrics with filtering and multiple visualization types
3. **Filtering Options**: No Filter, Migration Traffic Only, Exclude Migration Traffic, By ISP, By Node
4. **Visualization Types**: Blocking Probability, Availability, Link Utilization, Network Usage
5. **Dynamic Controls**: Time bucket slider for temporal analyses, Top N Links slider for link utilization (auto-adjusts to total link count)
6. **Timing Markers**: ISP Migration Start Times and Disaster Period
7. **Smart Caching**: Plots are cached automatically - switching between views or scenarios without changing parameters reuses cached plots for faster navigation

## 1. Setup and Imports

In [1]:
from pathlib import Path
import ast

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import analysis and visualization functions from modules
from simulador.analysis.scenario_comparison import (
    load_scenario_pair,
    apply_filter
)

from simulador.visualization.comparison_plots import (
    plot_blocking_probability_comparison,
    plot_availability_comparison,
    plot_link_utilization_comparison,
    plot_network_usage_comparison,
    visualize_isp_topology_comparison
)

# Style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 8)

print("✓ All modules loaded successfully!")

✓ All modules loaded successfully!


## 2. Interactive GUI

The GUI provides:
- Scenario selection dropdowns
- View mode toggle (Topology vs Statistics)
- Filtering options
- Visualization controls
- Timing marker toggles

In [2]:
def create_comparison_gui():
    """
    Create the main comparison GUI.
    """
    # Find available scenarios - only include those with both .pkl and .csv files
    output_path = Path("output")
    scenario_files = list(output_path.glob("*.pkl"))
    
    # Filter to only include scenarios that have both .pkl and corresponding df_*.csv
    valid_scenarios = []
    for pkl_file in scenario_files:
        scenario_name = pkl_file.stem
        csv_file = output_path / f"df_{scenario_name}.csv"
        if csv_file.exists():
            valid_scenarios.append(scenario_name)
    
    scenario_names = sorted(valid_scenarios)
    
    if len(scenario_names) < 2:
        print("Error: Need at least 2 valid scenarios (with both .pkl and df_*.csv files) in output/ folder")
        return
    
    # Create output widgets
    output_topology = widgets.Output()
    output_statistics = widgets.Output()
    
    # Scenario selection
    # Store all available scenarios
    all_scenario_names = scenario_names.copy()
    
    scenario1_dropdown = widgets.Dropdown(
        options=scenario_names,
        value=scenario_names[0],
        description='Scenario 1:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='250px')
    )
    
    # For scenario 2, exclude the currently selected scenario 1
    scenario2_options = [s for s in scenario_names if s != scenario1_dropdown.value]
    scenario2_dropdown = widgets.Dropdown(
        options=scenario2_options,
        value=scenario2_options[0] if scenario2_options else scenario_names[0],
        description='Scenario 2:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='250px')
    )
    
    # View mode
    view_mode = widgets.ToggleButtons(
        options=['Topology View', 'Statistics View'],
        value='Statistics View',
        description='View:',
        button_style='info',
        style={'description_width': '50px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Topology controls
    isp_dropdown = widgets.Dropdown(
        options=[],
        description='Select ISP:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='200px')
    )
    
    disaster_mode_checkbox = widgets.Checkbox(
        value=True,
        description='Disaster Mode (Remove Disaster Node)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='350px')
    )
    
    # Statistics controls
    filter_dropdown = widgets.Dropdown(
        options=['No Filter', 'Migration Traffic Only', 'Exclude Migration Traffic', 'By ISP', 'By Node'],
        value='No Filter',
        description='Filter:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='300px')
    )
    
    # Filter-specific controls
    filter_isp_dropdown = widgets.Dropdown(
        options=[],
        description='ISP:',
        style={'description_width': '40px'},
        layout=widgets.Layout(width='150px', display='none')
    )
    
    filter_node_dropdown = widgets.Dropdown(
        options=[],
        description='Node:',
        style={'description_width': '40px'},
        layout=widgets.Layout(width='150px', display='none')
    )
    
    viz_type_dropdown = widgets.Dropdown(
        options=['Blocking Probability Over Time', 'Availability Per Time Bucket', 
                 'Link Utilization', 'Network Usage'],
        value='Blocking Probability Over Time',
        description='Visualization:',
        style={'description_width': '90px'},
        layout=widgets.Layout(width='400px')
    )
    
    bucket_slider = widgets.IntSlider(
        value=10,
        min=1,
        max=50,
        step=1,
        description='Time Bucket:',
        style={'description_width': '90px'},
        layout=widgets.Layout(width='400px')
    )
    
    top_links_slider = widgets.IntSlider(
        value=15,
        min=5,
        max=50,  # Will be updated dynamically based on actual link count
        step=1,
        description='Top Links:',
        style={'description_width': '90px'},
        layout=widgets.Layout(width='400px', display='none')
    )
    
    show_migration_checkbox = widgets.Checkbox(
        value=True,
        description='Show ISP Migration Start Times',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    show_disaster_checkbox = widgets.Checkbox(
        value=True,
        description='Show Disaster Period',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px')
    )
    
    status_label = widgets.HTML(
        value="",
        layout=widgets.Layout(margin='10px 0')
    )
    
    # Cache for loaded data and computed plots
    cache = {
        'data': None, 
        'scenario1_name': None, 
        'scenario2_name': None,
        'plot_cache': {},  # Cache for computed plots
        'last_plot_params': None  # Parameters of last plot
    }
    
    # Update functions
    def load_scenarios():
        """Load scenarios if needed."""
        scenario1_name = scenario1_dropdown.value
        scenario2_name = scenario2_dropdown.value
        
        # Check cache
        if (cache['data'] is not None and 
            cache['scenario1_name'] == scenario1_name and 
            cache['scenario2_name'] == scenario2_name):
            return cache['data']
        
        # Load new data
        status_label.value = "<p style='color: blue;'>🔄 Loading scenarios...</p>"
        try:
            data = load_scenario_pair(scenario1_name, scenario2_name)
            cache['data'] = data
            cache['scenario1_name'] = scenario1_name
            cache['scenario2_name'] = scenario2_name
            
            # Update ISP dropdown
            isp_ids = [isp.isp_id for isp in data['scenario1'].lista_de_isps]
            isp_dropdown.options = isp_ids
            if isp_ids:
                isp_dropdown.value = isp_ids[0]
            
            # Update filter dropdowns
            filter_isp_dropdown.options = isp_ids
            if isp_ids:
                filter_isp_dropdown.value = isp_ids[0]
            
            all_nodes = sorted(set(data['df1']['src'].unique()) | set(data['df1']['dst'].unique()))
            filter_node_dropdown.options = all_nodes
            if all_nodes:
                filter_node_dropdown.value = all_nodes[0]
            
            status_label.value = "<p style='color: green;'>✅ Scenarios loaded successfully!</p>"
            return data
        except Exception as e:
            status_label.value = f"<p style='color: red;'>❌ Error loading scenarios: {str(e)}</p>"
            import traceback
            traceback.print_exc()
            return None
    
    def update_topology_view():
        """Update topology visualization."""
        with output_topology:
            clear_output(wait=True)
            
            data = load_scenarios()
            if data is None:
                return
            
            isp_id = isp_dropdown.value
            remove_disaster = disaster_mode_checkbox.value
            
            # Create cache key for topology view
            topo_cache_key = (
                'topology',
                scenario1_dropdown.value,
                scenario2_dropdown.value,
                isp_id,
                remove_disaster
            )
            
            # Check if we have cached plot
            if topo_cache_key in cache['plot_cache']:
                # Display cached plot
                fig = cache['plot_cache'][topo_cache_key]
                display(fig)
                return
            
            # Find ISPs in both scenarios
            isp1 = next((isp for isp in data['scenario1'].lista_de_isps if isp.isp_id == isp_id), None)
            isp2 = next((isp for isp in data['scenario2'].lista_de_isps if isp.isp_id == isp_id), None)
            
            if isp1 is None or isp2 is None:
                print(f"ISP {isp_id} not found in one or both scenarios")
                return
            
            # Show loading message
            print(f"⏳ Rendering ISP {isp_id} topology...")
            
            # Both scenarios share the same topology
            fig = visualize_isp_topology_comparison(
                isp1, isp2,
                data['scenario1'].topology,  # Shared topology
                data['disaster_node'],
                scenario1_dropdown.value,
                scenario2_dropdown.value,
                remove_disaster
            )
            
            # Cache the figure
            if fig is not None:
                cache['plot_cache'][topo_cache_key] = fig
            
            # Clear loading message and show plot
            clear_output(wait=True)
            if fig is not None:
                plt.show()
    
    def update_statistics_view():
        """Update statistics visualization."""
        with output_statistics:
            clear_output(wait=True)
            
            data = load_scenarios()
            if data is None:
                return
            
            # Apply filters
            filter_type = filter_dropdown.value
            filter_kwargs = {}
            
            if filter_type == 'By ISP':
                filter_kwargs['isp_id'] = filter_isp_dropdown.value
            elif filter_type == 'By Node':
                filter_kwargs['node'] = filter_node_dropdown.value
            
            df1_filtered = apply_filter(data['df1'], filter_type, **filter_kwargs)
            df2_filtered = apply_filter(data['df2'], filter_type, **filter_kwargs)
            
            # Check if filtered dataframes are empty
            if len(df1_filtered) == 0 and len(df2_filtered) == 0:
                print("No data available after applying filter")
                return
            
            # Get current parameters
            viz_type = viz_type_dropdown.value
            bucket_size = bucket_slider.value
            top_n_links = top_links_slider.value
            show_migration = show_migration_checkbox.value
            show_disaster = show_disaster_checkbox.value
            
            # Create cache key - include top_n for Link Utilization
            cache_key = (
                scenario1_dropdown.value,
                scenario2_dropdown.value,
                filter_type,
                filter_kwargs.get('isp_id'),
                filter_kwargs.get('node'),
                viz_type,
                bucket_size if viz_type in ['Blocking Probability Over Time', 'Availability Per Time Bucket'] else None,
                top_n_links if viz_type == 'Link Utilization' else None,
                show_migration,
                show_disaster
            )
            
            # Check if we have cached plot
            if cache_key in cache['plot_cache']:
                # Display cached plot
                fig = cache['plot_cache'][cache_key]
                display(fig)
                return
            
            # Show loading message
            print(f"⏳ Computing {viz_type}...")
            
            # Create plot
            fig, ax = plt.subplots(figsize=(16, 8))
            
            if viz_type == 'Blocking Probability Over Time':
                plot_blocking_probability_comparison(
                    ax, df1_filtered, df2_filtered, bucket_size,
                    scenario1_dropdown.value, scenario2_dropdown.value,
                    data['disaster_start'], data['disaster_end'],
                    data['migration_times'], show_migration, show_disaster
                )
            
            elif viz_type == 'Availability Per Time Bucket':
                plot_availability_comparison(
                    ax, df1_filtered, df2_filtered, bucket_size,
                    scenario1_dropdown.value, scenario2_dropdown.value,
                    data['disaster_start'], data['disaster_end'],
                    data['migration_times'], show_migration, show_disaster
                )
            
            elif viz_type == 'Link Utilization':
                # Count total unique links in dataframes to set slider max
                def count_total_links(df):
                    links = set()
                    for _, row in df.iterrows():
                        if pd.notna(row["caminho"]) and row["caminho"] != "":
                            try:
                                path = ast.literal_eval(row["caminho"])
                                for i in range(len(path) - 1):
                                    link = tuple(sorted([path[i], path[i + 1]]))
                                    links.add(link)
                            except Exception:
                                continue
                    return len(links)
                
                total_links = max(count_total_links(df1_filtered), count_total_links(df2_filtered))
                if total_links > 0:
                    top_links_slider.max = total_links
                
                plot_link_utilization_comparison(
                    ax, df1_filtered, df2_filtered,
                    scenario1_dropdown.value, scenario2_dropdown.value,
                    top_n=top_n_links
                )
            
            elif viz_type == 'Network Usage':
                plot_network_usage_comparison(
                    ax, df1_filtered, df2_filtered,
                    data['scenario1'].topology,
                    scenario1_dropdown.value, scenario2_dropdown.value,
                    data['disaster_start'], data['disaster_end'],
                    data['migration_times'], show_migration, show_disaster
                )
            
            plt.tight_layout()
            
            # Cache the figure before showing
            cache['plot_cache'][cache_key] = fig
            
            # Clear loading message and show plot
            clear_output(wait=True)
            plt.show()
    
    def on_view_mode_change(change):
        """Handle view mode change."""
        if change['new'] == 'Topology View':
            output_topology.layout.display = 'block'
            output_statistics.layout.display = 'none'
            topology_controls_row.layout.display = 'flex'
            statistics_controls_rows.layout.display = 'none'
            update_topology_view()
        else:
            output_topology.layout.display = 'none'
            output_statistics.layout.display = 'block'
            topology_controls_row.layout.display = 'none'
            statistics_controls_rows.layout.display = 'flex'
            update_statistics_view()
    
    def on_filter_change(change):
        """Handle filter type change."""
        filter_type = change['new']
        if filter_type == 'By ISP':
            filter_isp_dropdown.layout.display = 'block'
            filter_node_dropdown.layout.display = 'none'
        elif filter_type == 'By Node':
            filter_isp_dropdown.layout.display = 'none'
            filter_node_dropdown.layout.display = 'block'
        else:
            filter_isp_dropdown.layout.display = 'none'
            filter_node_dropdown.layout.display = 'none'
        
        if view_mode.value == 'Statistics View':
            update_statistics_view()
    
    def on_scenario1_change(change):
        """Handle scenario 1 selection change."""
        # Update scenario 2 options to exclude the selected scenario 1
        old_scenario2_value = scenario2_dropdown.value
        new_options = [s for s in all_scenario_names if s != scenario1_dropdown.value]
        scenario2_dropdown.options = new_options
        
        # Restore scenario 2 value if still valid, otherwise pick first available
        if old_scenario2_value in new_options:
            scenario2_dropdown.value = old_scenario2_value
        elif new_options:
            scenario2_dropdown.value = new_options[0]
        
        # Update view
        if view_mode.value == 'Topology View':
            update_topology_view()
        else:
            update_statistics_view()
    
    def on_scenario2_change(change):
        """Handle scenario 2 selection change."""
        # Update scenario 1 options to exclude the selected scenario 2
        old_scenario1_value = scenario1_dropdown.value
        new_options = [s for s in all_scenario_names if s != scenario2_dropdown.value]
        scenario1_dropdown.options = new_options
        
        # Restore scenario 1 value if still valid, otherwise pick first available
        if old_scenario1_value in new_options:
            scenario1_dropdown.value = old_scenario1_value
        elif new_options:
            scenario1_dropdown.value = new_options[0]
        
        # Update view
        if view_mode.value == 'Topology View':
            update_topology_view()
        else:
            update_statistics_view()
    
    def on_topology_param_change(change):
        """Handle topology parameter change."""
        if view_mode.value == 'Topology View':
            update_topology_view()
    
    def on_viz_type_change(change):
        """Handle visualization type change and update slider visibility."""
        viz_type = change['new']
        
        # Show/hide appropriate sliders based on visualization type
        if viz_type in ['Blocking Probability Over Time', 'Availability Per Time Bucket']:
            bucket_slider.layout.display = 'block'
            top_links_slider.layout.display = 'none'
        elif viz_type == 'Link Utilization':
            bucket_slider.layout.display = 'none'
            top_links_slider.layout.display = 'block'
        else:  # Network Usage
            bucket_slider.layout.display = 'none'
            top_links_slider.layout.display = 'none'
        
        # Update view
        if view_mode.value == 'Statistics View':
            update_statistics_view()
    
    def on_statistics_param_change(change):
        """Handle statistics parameter change."""
        if view_mode.value == 'Statistics View':
            update_statistics_view()
    
    # Attach observers
    view_mode.observe(on_view_mode_change, names='value')
    scenario1_dropdown.observe(on_scenario1_change, names='value')
    scenario2_dropdown.observe(on_scenario2_change, names='value')
    isp_dropdown.observe(on_topology_param_change, names='value')
    disaster_mode_checkbox.observe(on_topology_param_change, names='value')
    filter_dropdown.observe(on_filter_change, names='value')
    filter_isp_dropdown.observe(on_statistics_param_change, names='value')
    filter_node_dropdown.observe(on_statistics_param_change, names='value')
    viz_type_dropdown.observe(on_viz_type_change, names='value')
    bucket_slider.observe(on_statistics_param_change, names='value')
    top_links_slider.observe(on_statistics_param_change, names='value')
    show_migration_checkbox.observe(on_statistics_param_change, names='value')
    show_disaster_checkbox.observe(on_statistics_param_change, names='value')
    
    # Layout
    scenario_row = widgets.HBox([scenario1_dropdown, scenario2_dropdown])
    view_row = widgets.HBox([view_mode])
    
    topology_controls_row = widgets.HBox(
        [isp_dropdown, disaster_mode_checkbox],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    filter_row = widgets.HBox([filter_dropdown, filter_isp_dropdown, filter_node_dropdown])
    viz_row = widgets.HBox([viz_type_dropdown])
    sliders_row = widgets.HBox([bucket_slider, top_links_slider])
    toggles_row = widgets.HBox([show_migration_checkbox, show_disaster_checkbox])
    
    statistics_controls_rows = widgets.VBox(
        [filter_row, viz_row, sliders_row, toggles_row],
        layout=widgets.Layout(display='flex', margin='5px 0')
    )
    
    gui = widgets.VBox([
        scenario_row,
        view_row,
        topology_controls_row,
        statistics_controls_rows,
        status_label,
        output_topology,
        output_statistics
    ])
    
    # Initialize
    update_statistics_view()
    
    return gui

## 3. Launch GUI

In [3]:
# Create and display the GUI
gui = create_comparison_gui()
display(gui)